# recommendMe

In [1]:
from flask import Flask
from database import DataBase
import utils
import RecEng

### Set up

In [2]:
# Key for IMDb api
API_KEY = "c30070156d8d7a150ea066f8094a2013"

# Our database
database = DataBase("vk_users")

# Recommendation class
recommender = RecEng.Recommender(database)

### Downloading new users

In [ ]:
utils.get_data(68024287, 1000000, database, "User_Info", API_KEY)

In [3]:
utils.download_avatars(database, name="photo_200")

## Тестим

In [5]:
id_ = 366562392

In [6]:
for x in database.get_one(id_, "User_Info")['movies']:
    print(database.get_one(x, "Movie_Info")['title'])

Только сильнейшие
Последний самурай
Удачи, Чак!
Секрет Анастасии


In [7]:
for movie in recommender.ratings_rec(id_):
    print(movie['title'])

Добро пожаловать в рай
Элвис и Никсон
Три цвета: Белый
Серена
Формула любви
Горько!
Руины
Все умрут, а я останусь
Первый эшелон
Угнать за 60 секунд


In [8]:
for movie in recommender.groups_rec(id_):
    print(movie['title'])

Первое признание
Прогулка
Звездные Войны: Откровения
Служебный роман
Девчата
Иван Васильевич меняет профессию
Полицейский – гладиатор
Унесенные ветром
Ирония судьбы, или С легким паром!
Служебный роман
Куда приводят мечты
Легенды осени
Мирный воин
Властелин Колец: Возвращение Бомжа
Звонок 2
Заложница 3
Неоспоримый 4
5-я волна
Кухня. Последняя битва
Аватар
Казанова
Титаник
Мемуары гейши


## Web

In [94]:
app = Flask("recApp")

In [96]:
@app.route("/getRecs/<int:user_id>")
def get_recs(user_id):
    user = user_info.find_one({"_id":user_id})
    recs = []
    if user:
        matr, users, movies = build_matrix(build_type="ratings")
        recs = ratings_rec(matrix, users[user_id], users, movies)
    else:
        vk_user = vk.get_users_info([user_id])[str(user_id)]
        if 'movies' in vk_user and len(vk_user['movies']) != 0:
            unchecked = []
            for movie in vk_user['movies']:
                m = movie_info.find_one({"title":movie}) 
                if m:
                    recs.append(m)
                else:
                    unchecked.append(movie)
            for movie in unchecked:
                m = get_movie_info(movie, API_KEY)
                if m != []:
                    m["_id"] = m["id"]
                    del m["id"]
                    recs.append(m)
                    try:
                        movie_info.insert_one(m)
                    except:
                        1+1
    return json.dumps(recs)

In [97]:
app.run(port=8888)